# DACON "AI야, 진짜 뉴스를 찾아줘!" - 북극506번지  
  
주 언어 : Pytorch

## Install Libraries

In [1]:
# Hugging Face의 Transformers
!pip install transformers

     |████████████████████████████████| 1.5MB 14.4MB/s 
     |████████████████████████████████| 890kB 41.9MB/s 
     |████████████████████████████████| 2.9MB 44.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=2ac1eee27ccf1d14427edc0b1e57829893a636d769362f38330900980880c8f3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# 데이터 전처리를 위한 라이브러리
import numpy as np
import pandas as pd
import warnings
import os, sys
import random
import time
import datetime
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_row', 500)

# 모델링을 위한 라이브러리
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertTokenizerFast
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

## Device Setting

In [4]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('사용 가능한 %d개의 GPU가 존재합니다.' % torch.cuda.device_count())
    print('{} GPU를 사용할 예정입니다.'.format(torch.cuda.get_device_name(0)))
else:
    device = torch.device("cpu")
    print('사용 가능한 GPU가 존재하지 않으므로, CPU를 사용할 예정입니다.')

사용 가능한 1개의 GPU가 존재합니다.
Tesla V100-SXM2-16GB GPU를 사용할 예정입니다.


## Data Loading

In [5]:
# 학습 데이터 로드
train = pd.read_csv('/content/gdrive/My Drive/NH공모전/data/news_train.csv', encoding = 'utf-8-sig')

## Data preprocessing

### Ord  
  
뉴스 기사 내에서 문장의 위치를 나타내는 변수.  
> `ordp_s` 생성  
  = (ord / max_ord) * 100. 뉴스 기사 내에서 문장의 상대적인 위치를 나타내는 변수.

In [6]:
max_ord = train.groupby('n_id').max(ord).ord
train = pd.merge(train, max_ord, on='n_id')
train['ordp'] = train.ord_x/train.ord_y
train['ordp_s'] = (train.ordp*100).astype(int).astype(str)

### Content  
  
뉴스 기사 내용을 의미하는 변수.  
특수문자, 조사 등은 제거하지 않음.  

### Title  

뉴스 기사 제목을 의미하는 변수.
특수문자, 조사 등은 제거하지 않음. 

### Dataset 
  
> Input = `content` + '  ' + `ordp_s` + '  ' + `title`

In [7]:
# Input sentences 생성
sentences = train['content'] + ' ' + train['ordp_s'] + ' ' + train['title']

# Input sentences를 BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

# Label 추출
labels = train['info'].values

# BERT의 토크나이저로 문장을 토큰으로 분리
# bert-base-multilingual-cased pretrained tokenizer 사용
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors



[CLS] [이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도 25 [마감]코스닥 기관 678억 순매도 [SEP]
['[CLS]', '[', '이', '##데', '##일', '##리', 'MA', '##R', '##K', '##ET', '##PO', '##IN', '##T', ']', '15', ':', '32', '현재', '코', '##스', '##닥', '기', '##관', '678', '##억', '순', '##매', '##도', '25', '[', '마', '##감', ']', '코', '##스', '##닥', '기', '##관', '678', '##억', '순', '##매', '##도', '[SEP]']


In [8]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 100

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,    164,   9638,  28911,  18392,  12692,  27277,  11273,
        11733,  52338,  93520,  27128,  11090,    166,  10208,    131,
        10842,  26565,   9812,  12605, 118770,   8932,  20595,  69015,
        91837,   9462, 100372,  12092,  10258,    164,   9246, 105197,
          166,   9812,  12605, 118770,   8932,  20595,  69015,  91837,
         9462, 100372,  12092,    102,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0])

In [9]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 어텐션 마스크 0인 부분은 계산이 이루어지지 않으므로 속도 향상 가능
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [10]:
# 데이터를 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2020, 
                                                                                    test_size=0.2)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2020, 
                                                       test_size=0.2)

# 데이터를 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9689,  21155,  14040,  83685,   9566,  32159,  25805,   9513,
          9566,  14423,  37568,   9684,  68055,    119,   9284,  37824,  17322,
           110,  66982,    119,  11978,   2650,    117,   9309,  37388,  43022,
         33188,  21928,   9736,  21928,  10530,    100,   9356,  15891,   9098,
        106249,  27654,   9657,  40419,  88236,   8987,  20626,  18471,   9067,
         20479,    100,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0])
tensor(1)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1

In [11]:
# 배치 사이즈
batch_size = 32

# DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## Modeling

In [12]:
# 분류를 위한 BERT 모델 생성
# bert-base-multilingual-cased model 사용
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [13]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 3

# 총 훈련 스텝 = 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 변화시키는 스케줄러 생성 (선형적으로 감소)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [14]:
# 정확도 계산 함수
def calculate_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [15]:
# 랜덤 시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = calculate_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 3 ========
Training...
  Batch   500  of  2,969.    Elapsed: 0:01:41.
  Batch 1,000  of  2,969.    Elapsed: 0:03:23.
  Batch 1,500  of  2,969.    Elapsed: 0:05:04.
  Batch 2,000  of  2,969.    Elapsed: 0:06:44.
  Batch 2,500  of  2,969.    Elapsed: 0:08:25.

  Average training loss: 0.0393
  Training epcoh took: 0:09:59

Running Validation...
  Accuracy: 0.9959
  Validation took: 0:00:40

======== Epoch 2 / 3 ========
Training...
  Batch   500  of  2,969.    Elapsed: 0:01:40.
  Batch 1,000  of  2,969.    Elapsed: 0:03:20.
  Batch 1,500  of  2,969.    Elapsed: 0:05:01.
  Batch 2,000  of  2,969.    Elapsed: 0:06:41.
  Batch 2,500  of  2,969.    Elapsed: 0:08:21.

  Average training loss: 0.0096
  Training epcoh took: 0:09:56

Running Validation...
  Accuracy: 0.9982
  Validation took: 0:00:40

======== Epoch 3 / 3 ========
Training...
  Batch   500  of  2,969.    Elapsed: 0:01:40.
  Batch 1,000  of  2,969.    Elapsed: 0:03:20.
  Batch 1,500  of  2,969.    Elapsed: 0:0

In [16]:
# 모델 저장
torch.save(model.state_dict(), './gdrive/My Drive/NH공모전/혜린의 모델찾기대작전/NH_final_model.pt')